In [ ]:
# we need the following libraries, so let's install them
!pip install boto3
!pip install torchxrayvision
!pip install SimpleITK
!pip install radiomics
!pip install pyradiomics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing libraries
import boto3
import numpy as np
import pandas as pd
from glob import glob
import os
import skimage
import torchxrayvision as xrv
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
path = os.getcwd()  # this line is used to get the root path
name_of_folder = 'MIMIC'   # defining the name of folder to save mimic datasets

# defining subfolders to save images and their targets (labels)
path = path+'/sample_data' + '/' + name_of_folder
path_targets = path+'/'+'targets'
path_images = path+'/'+'images'
path_codes = path+'/'+'codes'


# now we make these folders
try:
  os.makedirs(path) # make folder mimic
except:
  pass

try:
  os.makedirs(path_targets)
except:
  pass

try:
 os.makedirs(path_images )
except:
 pass

try:
  os.makedirs(path_codes)
except:
  pass

In [ ]:
# connecting to the bucket of image data in S3
aws_access_key_id = '************************'
aws_secret_access_key = '************************'
region='us-east-1'
Bucket = 'um2ii-datasets'
s3 = boto3.client(service_name = 's3',
                    region_name = region,
                    aws_access_key_id = aws_access_key_id,
                    aws_secret_access_key = aws_secret_access_key)

In [ ]:
os.chdir(path_targets) # we select folder targete (labels) to save targets of mimic in aws
s3.download_file(Bucket=Bucket,Filename='mimic-cxr-2.0.0-negbio.csv',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/mimic-cxr-2.0.0-negbio.csv')
s3.download_file(Bucket=Bucket,Filename='mimic-cxr-2.0.0-split.csv',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/mimic-cxr-2.0.0-split.csv')
s3.download_file(Bucket=Bucket,Filename='patients.csv',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/patients.csv')
s3.download_file(Bucket=Bucket,Filename='SHA256SUMS.txt',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/SHA256SUMS.txt')
s3.download_file(Bucket=Bucket,Filename='mimic-cxr-2.0.0-chexpert.csv',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/mimic-cxr-2.0.0-chexpert.csv')
s3.download_file(Bucket=Bucket,Filename='mimic-cxr-2.0.0-metadata.csv',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/mimic-cxr-2.0.0-metadata.csv')
s3.download_file(Bucket=Bucket,Filename='admissions.csv',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/admissions.csv')
s3.download_file(Bucket=Bucket,Filename='LICENSE.txt',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/LICENSE.txt')
s3.download_file(Bucket=Bucket,Filename='README',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/README')
s3.download_file(Bucket=Bucket,Filename='metadata.csv',Key='MIMIC/mimic-cxr-jpg-small/2.0.0/metadata.csv')

In [ ]:
data = pd.read_csv('metadata.csv') # we visualize one  of these target files
data.head(1)

,path,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,Race,Ethnicity,...,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,CHEST (PA AND LAT),PA,CHEST (PA AND LAT),postero-anterior,WHITE,NOT Hispanic/latino,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
N = data.shape[0] # number of images

In [ ]:
all_names =list( data.iloc[:,0].values) # getting the paths of all images (the first column of the target datas)
name = data.iloc[0,0]


In [ ]:
for n in tqdm(range(N)):
  name = all_names[n] # the address of image number n
  split_ = name.split('/') # these 3 lines are used to get the name of image and also build the name of the address that the images ig going  to be saved
  image_name = split_[-1]
  pth = path_images + '/' + split_[0] + '/' + split_[1] + '/' + split_[2] + '/' + split_[3]  # this is the folder that image n will be saved into
  try:
    os.makedirs(pth) # before saving image, we need to make its folder
  except:
      pass
  os.chdir(pth) # we select that folder (path)
  s3.download_file(Bucket=Bucket,Filename=image_name,Key='/MIMIC/mimic-cxr-jpg-small/2.0.0' + name)


100%|██████████| 377095/377095 [20:33:22<00:00,  5.10it/s]


In [ ]:
# reading the labels of MIMIC dataset
data=pd.read_csv('/content/sample_data/MIMIC/targets/metadata.csv')
data.head(2)

,path,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,Race,Ethnicity,...,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,CHEST (PA AND LAT),PA,CHEST (PA AND LAT),postero-anterior,WHITE,NOT Hispanic/latino,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,10000032,50414267,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,CHEST (PA AND LAT),LATERAL,CHEST (PA AND LAT),lateral,WHITE,NOT Hispanic/latino,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# printing the unique races from MIMIC dataset
pd.unique(data['Race'])

array(['WHITE', nan, 'BLACK/AFRICAN AMERICAN', 'WHITE - RUSSIAN',
       'UNKNOWN', 'PORTUGUESE', 'OTHER', 'BLACK/CAPE VERDEAN', 'ASIAN',
       'ASIAN - CHINESE', 'HISPANIC/LATINO - GUATEMALAN',
       'WHITE - BRAZILIAN', 'HISPANIC/LATINO - PUERTO RICAN',
       'HISPANIC OR LATINO', 'WHITE - OTHER EUROPEAN',
       'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER', 'UNABLE TO OBTAIN',
       'PATIENT DECLINED TO ANSWER', 'BLACK/CARIBBEAN ISLAND',
       'WHITE - EASTERN EUROPEAN', 'BLACK/AFRICAN',
       'ASIAN - SOUTH EAST ASIAN', 'HISPANIC/LATINO - CUBAN',
       'AMERICAN INDIAN/ALASKA NATIVE', 'HISPANIC/LATINO - DOMINICAN',
       'ASIAN - ASIAN INDIAN', 'HISPANIC/LATINO - COLUMBIAN',
       'HISPANIC/LATINO - CENTRAL AMERICAN', 'HISPANIC/LATINO - HONDURAN',
       'HISPANIC/LATINO - SALVADORAN', 'ASIAN - KOREAN',
       'MULTIPLE RACE/ETHNICITY', 'SOUTH AMERICAN',
       'HISPANIC/LATINO - MEXICAN'], dtype=object)